# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 11:31AM,249696,102,7154975,"Senseonics, Incorporated",Released
1,Oct 31 2022 11:27AM,249695,10,129874,"Virtue Rx, LLC",Released
2,Oct 31 2022 11:27AM,249695,10,129904,"Virtue Rx, LLC",Released
3,Oct 31 2022 11:24AM,249694,19,MSP215365,"Methapharm, Inc.",Released
4,Oct 31 2022 11:20AM,249693,16,8715335,Sartorius Bioprocess Solutions,Released
5,Oct 31 2022 11:20AM,249692,19,ADV80007174,"AdvaGen Pharma, Ltd",Released
6,Oct 31 2022 11:20AM,249692,19,ADV80007175,"AdvaGen Pharma, Ltd",Released
7,Oct 31 2022 11:20AM,249692,19,ADV80007176,"AdvaGen Pharma, Ltd",Released
8,Oct 31 2022 11:20AM,249692,19,ADV80007177,"AdvaGen Pharma, Ltd",Released
9,Oct 31 2022 11:20AM,249692,19,ADV80007178,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,249692,Released,19
25,249693,Released,1
26,249694,Released,1
27,249695,Released,2
28,249696,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249692,NaN,19.0
249693,NaN,1.0
249694,NaN,1.0
249695,NaN,2.0
249696,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249646,8.0,1.0
249651,16.0,2.0
249667,20.0,0.0
249670,0.0,49.0
249671,0.0,63.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249646,8,1
249651,16,2
249667,20,0
249670,0,49
249671,0,63


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249646,8,1
1,249651,16,2
2,249667,20,0
3,249670,0,49
4,249671,0,63


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249646,8,1
1,249651,16,2
2,249667,20,
3,249670,,49
4,249671,,63


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated"
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC"
3,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc."
4,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions
5,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd"
24,Oct 31 2022 11:16AM,249691,12,Hush Hush
25,Oct 31 2022 11:16AM,249690,12,Hush Hush
26,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc."
27,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc."
30,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated",,1
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC",,2
2,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc.",,1
3,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,,1
4,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",,19
5,Oct 31 2022 11:16AM,249691,12,Hush Hush,,1
6,Oct 31 2022 11:16AM,249690,12,Hush Hush,1,
7,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",,1
8,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc.",,3
9,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated",1,
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC",2,
2,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc.",1,
3,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1,
4,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19,
5,Oct 31 2022 11:16AM,249691,12,Hush Hush,1,
6,Oct 31 2022 11:16AM,249690,12,Hush Hush,,1
7,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",1,
8,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc.",3,
9,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated",1,
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC",2,
2,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc.",1,
3,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1,
4,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated",1.0,NaN
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC",2.0,NaN
2,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc.",1.0,NaN
3,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1.0,NaN
4,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 11:31AM,249696,102,"Senseonics, Incorporated",1.0,0.0
1,Oct 31 2022 11:27AM,249695,10,"Virtue Rx, LLC",2.0,0.0
2,Oct 31 2022 11:24AM,249694,19,"Methapharm, Inc.",1.0,0.0
3,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1.0,0.0
4,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3245803,172.0,61.0
102,249696,1.0,0.0
12,499381,1.0,1.0
16,249693,1.0,0.0
19,1248401,45.0,8.0
20,499355,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3245803,172.0,61.0
1,102,249696,1.0,0.0
2,12,499381,1.0,1.0
3,16,249693,1.0,0.0
4,19,1248401,45.0,8.0
5,20,499355,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,172.0,61.0
1,102,1.0,0.0
2,12,1.0,1.0
3,16,1.0,0.0
4,19,45.0,8.0
5,20,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,172.0
1,102,Released,1.0
2,12,Released,1.0
3,16,Released,1.0
4,19,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,19,20
Status,,,,,,
Executing,61.0,0.0,1.0,0.0,8.0,0.0
Released,172.0,1.0,1.0,1.0,45.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,19,20
0,Executing,61.0,0.0,1.0,0.0,8.0,0.0
1,Released,172.0,1.0,1.0,1.0,45.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,19,20
0,Executing,61.0,0.0,1.0,0.0,8.0,0.0
1,Released,172.0,1.0,1.0,1.0,45.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()